In [1]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import pendulum
from PIL import Image
import pickle

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Reshape, Layer, Activation, GlobalAveragePooling2D, Add, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D 
from keras.utils import np_utils
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import ConvLSTM2D
from keras.layers import BatchNormalization

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn import metrics

from tensorflow import keras
import keras.backend as K

from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Normalization, Input

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from tensorflow.keras.layers import Dense, Flatten, Masking, Dropout
from tensorflow.keras.layers import GRU, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.metrics import Recall, Precision

from keras.layers.convolutional import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D

In [4]:
class CustomConv2D(Layer):
    def __init__(self, n_filters, kernel_size, n_strides, padding="same", **kwargs):
        super(CustomConv2D, self).__init__()
        self.conv = Conv2D(filters = n_filters,
                           kernel_size = kernel_size,
                           activation='relu', 
                           strides=n_strides, 
                           padding=padding)
            
            #self.batch_norm = BatchNormalization()
        
    def call(self, x):
        x = self.conv(x)
       #x = self.batch_norm(x)
            
        return x

In [5]:
class ResidualBlock(Layer):
        def __init__(self, n_channels, n_strides=1):
            super(ResidualBlock, self).__init__() 
            
            self.dotted = (n_strides != 1)
            
            self.custom_conv_1= CustomConv2D(n_channels, 3, n_strides, padding="same")
            self.batch_norm1 = BatchNormalization(axis=3)
            self.custom_conv_2= CustomConv2D(n_channels, 3, 1, padding="same")
            self.batch_norm2 = BatchNormalization(axis=3)
            
            self.activation = Activation('relu')
            if self.dotted:
                self.custom_conv_3 = CustomConv2D(n_channels, 1, n_strides)
            
        
        def call(self, input):
            x = self.custom_conv_1(input)
            x = self.batch_norm1(x)
            x = self.custom_conv_2(x)
            x = self.batch_norm2(x)
            
            if self.dotted:
                x_add = self.custom_conv_3(input)
                x_add = Add()([x, x_add])
            else: 
                    x_add = Add()([x, input])
                    
            return self.activation(x_add)
    
            

In [6]:
#Coding ResNet from scratch, as seen in the paper 
class ResNet18(Model):
    def __init__(self, **kwargs):
        super(ResNet18, self).__init__()
        
        #conv layer
        self.conv_1 = CustomConv2D(64, 7, 2, padding='same')
        self.batch_norm = BatchNormalization(axis=3)
        self.max_pool = MaxPooling2D((4, 2))
        
        #Res layers
        self.conv_2_1 = ResidualBlock(64, 2)
        self.conv_2_2 = ResidualBlock(64)
        
        self.conv_3_1 = ResidualBlock(128, 2)
        self.conv_3_2 = ResidualBlock(128)
        
        self.conv_4_1 = ResidualBlock(256, 2)
        self.conv_4_2 = ResidualBlock(256)
        
        self.conv_5_1 = ResidualBlock(512, 2)
        self.conv_5_2 = ResidualBlock(512)
        
        #Global average pooling / Flatten / Dense / Output layer 
        
        self.global_pool = GlobalAveragePooling2D()
        self.flatten = Flatten()
        self.dense_32 = Dense(32,activation='relu')
        self.dense_7 = Dense(7, activation='softmax')
        
    def call(self, x): 
        #conv layer
        x = self.conv_1(x)
        x = self.batch_norm(x)
        x = self.max_pool(x)
        
        #res layers
        x = self.conv_2_1(x)
        x = self.conv_2_2(x)
        
        x = self.conv_3_1(x)
        x = self.conv_3_2(x)
        
        x = self.conv_4_1(x)
        x = self.conv_4_2(x)
        
        x = self.conv_5_1(x)
        x = self.conv_5_2(x)
        
        #output shenanigans 
        x = self.global_pool(x)
        x = self.flatten(x)
        x = self.dense_32(x)
        
        
        return self.dense_7(x)

In [7]:
model = ResNet18()

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])